In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rosbag
import rospy
# import time
# from scipy.signal import savgol_filter
# import os
# from datetime import datetime
%matplotlib notebook

In [ ]:
f_name = 'drive_18Mar_1.bag' ## after fixes on 9MAR
bag = rosbag.Bag(f_name)

In [ ]:
mapdtype = np.dtype({"names":["x", "y", "z", "rosSec", "rosNsec","rosSysTme", "run_time", 'msg_time'],
                     "formats":[np.float32, np.float32, np.float32, np.float32, np.float32, np.float32, np.float32, np.float64]})

xpos = []
ypos = []
zpos = []
t_sec = []
t_nsec = []
msgtme = []
ndx = 0
for topic, msg, ts in bag.read_messages(topics=['/detector_data', '/trajectory_node_list', '/gx5/imu/data', '/map']):
#     print topic
    if topic == '/detector_data':
        pass
    if topic == '/trajectory_node_list':
        hold = msg
#         print type((ts.to_sec()))
#         break
        if len(hold.markers[2].points)-ndx != 0:
            
            for i in range(len(hold.markers[2].points))[ndx:]:
                x = hold.markers[2].points[i].x
                y = hold.markers[2].points[i].y
                z = hold.markers[2].points[i].z
                xpos.append(x)
                ypos.append(y)
                zpos.append(z)
                t_sec.append(hold.markers[2].header.stamp.secs)
                t_nsec.append(hold.markers[2].header.stamp.nsecs)
                msgtme.append(rospy.Time(hold.markers[2].header.stamp.secs, hold.markers[2].header.stamp.nsecs).to_sec())
            ndx = len(hold.markers[2].points)
        
# for i in range(len(hold.markers[2].points)):
    
#     x = hold.markers[2].points[i].x
#     y = hold.markers[2].points[i].y
#     z = hold.markers[2].points[i].z
#     xpos.append(x)
#     ypos.append(y)
#     zpos.append(z)

s = np.asarray(t_sec)
ns = np.asarray(t_nsec)/1e9
runtm = s+ns
runtm = runtm-runtm[0]

path = np.zeros(len(xpos), dtype=mapdtype)
path["x"] =xpos
path["y"] = ypos
path["z"] = zpos
path["rosSec"] = s
path["rosNsec"] = ns
path['rosSysTme'] = s+ns
path["run_time"] = runtm
path['msg_time'] = msgtme
mask = path["run_time"]>0.0
path = path[mask]

In [ ]:
path['msg_time'][:20]/1e9

In [ ]:
'''
Reads the /radrates topic and returns a numpy structured away with total count rate and time stamp data
'''
raddtype = np.dtype({"names":["det1", "det2", "det3", "det4", "det5",'total', "cr_norm", "x", "y", "rosSec", "rosNsec",'rosSysTme', "run_time", "msg_time"],
                     "formats":[np.int16, np.int16, np.int16, np.int16, np.int16, np.int16, np.float32, np.float32, np.float32, np.float32, np.float32, np.float32, np.float32, np.float64]})

# bag2 = rosbag.Bag('arraytopic.bag')
det1 = []
det2 = []
det3 = []
det4 = []
det5 = []
total = []
rsec = []
rnsec = []
msg_times = []
for topic, msg, ts in bag.read_messages(topics=['/radrates']):
    det1.append(msg.cr[0])#np.sum(msg.cr[:3])+msg.cr[4])
    det2.append(msg.cr[1])
    det3.append(msg.cr[2])
    det4.append(msg.cr[3])
    det5.append(msg.cr[4])
    total.append(msg.cr[5])
    rsec.append(msg.header.stamp.secs)
    rnsec.append(msg.header.stamp.nsecs)
    t = rospy.Time(msg.header.stamp.secs, msg.header.stamp.nsecs).to_sec() #needs to be 64bit
    msg_times.append(t)
# print det1, total

arr = np.asarray(total)
normCR = np.divide(arr, arr.max(), dtype=np.float32)

rs = np.asarray(rsec)
rns = np.asarray(rnsec)/1e9
Rruntm = rs+rns
Rruntm = Rruntm-Rruntm[0]

radmap = np.zeros(len(arr), dtype=raddtype)
radmap["det1"] = det1
radmap["det2"] = det2
radmap["det3"] = det3
radmap["det4"] = det4
radmap["det5"] = det5
radmap["total"] = total # total of counts published every 10Hz
radmap['cr_norm'] = normCR
# radmap["x"] = ypos
# radmap["y"] = zpos
radmap["rosSec"] = rs
radmap["rosNsec"] = rns
radmap['rosSysTme'] = rs+rns
radmap["run_time"] =Rruntm
radmap['msg_time'] = msg_times

In [ ]:
radmap['msg_time'][2]

In [ ]:
c=0
rad_x = []
rad_y = []
n = 0
# for d in radmap["run_time"]:
for d in radmap["msg_time"]:
    n+=1
    if c >=len(path):
        rad_x.append(0)
        rad_y.append(0)
#     elif d < path['run_time'][c]:  
    elif d < path['msg_time'][c]:
        rad_x.append(0)
        rad_y.append(0)
    else:
        rad_x.append(path['x'][c])
        rad_y.append(path['y'][c])
        c+=1

In [ ]:
len(rad_x)

In [ ]:
aa = pd.Series(rad_x)
bb = pd.Series(rad_y)

aa.replace(0.0, np.NaN, inplace=True)
bb.replace(0.0, np.NaN, inplace=True)

rad_x_full = aa.interpolate()
rad_y_full = bb.interpolate()

In [ ]:
# a = np.asarray(rad_x_full)
# b = np.asarray(rad_y_full)
# a.round(decimals=4)
# b.round(decimals=4)

# mask = np.zeros(len(rad_x_full), dtype=bool)

# radinput = radmap['total'][:]
# shift = 0
# stitch = np.linspace(0+shift, len(rad_x_full), num=len(rad_x_full), endpoint=False, dtype=int)
# mask[stitch] = 1

In [ ]:
len(rad_x_full), len(radmap)

In [ ]:
fig, ax1 = plt.subplots(figsize=(10,6))
line, = ax1.plot(-1*rad_y_full, rad_x_full)
scat = ax1.scatter(-1*rad_y_full, rad_x_full,  c=radmap['total'], s=(radmap['total']/10)**2.2, cmap='viridis', alpha=.5)
# fig.colorbar(scat, ax=ax1)

cbar = plt.colorbar(scat)
# cbar.ax1.set_yticklabels(['0','1','2','>3'])
cbar.set_label('CPS', rotation=0)


# ax1.set_xlim(15, -2)
plt.title("Localization of 20uCi Cs-137 in Hallway")
ax1.set_xlabel("X Position")
ax1.set_ylabel("Y Position")
# plt.savefig('localization_map.pdf', dpi = 800)
plt.show()

I need to sync the ros time, the mapping ros time starts before the detectors. With everything starting at once, there is a significant delay between map start and det data publish. Once arrays are made, I need to mask map data before the first detector ros time value. 

In [ ]:
radmap[["rosSec", "rosNsec",'rosSysTme', "run_time"]][:4], path[["rosSec", "rosNsec",'rosSysTme', "run_time"]][:4]

In [ ]:
radmap[0], path[0]

In [ ]:
plt.plot(path['msg_time'], label="route")
plt.plot(radmap["msg_time"], label="dets")
plt.legend()
plt.show()

In [ ]:
path['msg_time'][:100]